In [ ]:
#!pip install kaggle

In [ ]:
%tensorflow_version 2.x
%load_ext tensorboard

# Get the data

upload the dinos.txt file into /content directory from kaggle dataset

In [ ]:
!mkdir /content/.kaggle

In [ ]:
# get the keys from your kaggle account information
# my account > API > Create New API Token
# get the content of the file 
import json
token = {"username":"jeankunz","key":"94c94aa22b3a009d7e6e48bbd45bd6ee"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
%%bash

if [ ! -f /content/dinos.txt ]; then
    mkdir /root/.kaggle
    cp /content/.kaggle/kaggle.json /root/.kaggle/
    cat /root/.kaggle/kaggle.json
    kaggle config set -n path -v{/content}
    kaggle datasets download -d swimmingwhale/dinosaur-island  -p /content
    
    unzip -o /content/dinosaur-island.zip -d /content
fi


# Data preparation

In [ ]:
data = open('/content/dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size)) 

In [ ]:
dino_list = list(data.split('\n'))

The characters are a-z (26 characters) plus the "\n" (or newline character), which in this assignment plays a role similar to the <EOS> (or "End of sentence") token we had discussed in lecture, only here it indicates the end of the dinosaur name rather than the end of a sentence. In the cell below, we create a python dictionary (i.e., a hash table) to map each character to an index from 0-26. We also create a second python dictionary that maps each index back to the corresponding character character. This will help you figure out what index corresponds to what character in the probability distribution output of the softmax layer. Below, char_to_ix and ix_to_char are the python dictionaries.

In [ ]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

In [ ]:
import tensorflow as tf
import numpy as np

from absl import logging

In [ ]:
 # Build list of all dinosaur names (training examples).
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip()+"\n" for x in examples]

# Shuffle list of all dinosaur names
np.random.seed(0)
np.random.shuffle(examples)

In [ ]:
examples[:5]

In [ ]:
def string_to_ixs(str):
    return [char_to_ix[c] for c in str]

def ixs_to_string(ixs):
    chars= [ix_to_char[ix] for ix in ixs]
    return "".join(chars)

ixs = string_to_ixs("hello")
assert 'hello'==ixs_to_string(ixs)

'''
def string_to_X(str, max_len,char_to_ix):
    nb_zeros_to_add = max_len - (len(str)+1)    
    X = [char_to_ix[ch] for ch in str] 
    X_padded = X + [0]*nb_zeros_to_add

    return X_padded

def string_to_Y(str, max_len,char_to_ix):
    # we add 1 to maxlen because we always add a last char of \n
    max_len+=1
    nb_zeros_to_add = max_len - (len(str)+1)
    Y = [char_to_ix[ch] for ch in str] + [char_to_ix["\n"]]
    Y_padded = Y + [0]*nb_zeros_to_add
  
    return Y_padded
'''
def create_x_y(str):
    #nb_zeros_to_add = max_len - (len(str)+1)
    str_ix = [char_to_ix[ch] for ch in str]
    X = str_ix[:-1]
    Y = str_ix[1:]
    return X,Y


create_x_y("hello\n")


In [ ]:
#example_ds = tf.data.Dataset.from_tensor_slices(examples)
#next(iter(example_ds.take(1)))

In [ ]:
dino_max_len = len(max(examples, key=len))+1
X = []
Y = []
for dino in examples:
    x,y = create_x_y(dino)
    X.append(x)
    Y.append(y)


In [ ]:
X[:2], Y[:2]

In [ ]:
padded_X = tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=dino_max_len,padding='post')
padded_Y = tf.keras.preprocessing.sequence.pad_sequences(Y,maxlen=dino_max_len,padding='post')

In [ ]:
# we don't padd because we want the model to be dynamic, with no fixed length
data_ds = tf.data.Dataset.from_tensor_slices((padded_X,padded_Y))

RNN model

![Texte alternatif…](https://www.tensorflow.org/tutorials/text/images/text_generation_training.png)

Diff between vanilla RNN, GRU, LSTM: https://medium.com/@saurabh.rathor092/simple-rnn-vs-gru-vs-lstm-difference-lies-in-more-flexible-control-5f33e07b1e57

https://towardsdatascience.com/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9

In [ ]:
seq_len = dino_max_len
embedding_size = 32
rnn_units= 128
batch_size = 64

In [ ]:
seq_len

In [ ]:
def build_model(embedding_size, rnn_units,vocab_size):
    # No constraint on batch_size and seq_len in model
    inputs = tf.keras.Input(batch_shape=(None, None))
    x = tf.keras.layers.Embedding(vocab_size,embedding_size)(inputs)
    # stateful is useful when we want to keep the state (s, in case of lstm) (not the weight) between samples, 
    #like when we learn from a very long sequence (a book for instance) that we split.
    x = tf.keras.layers.GRU(rnn_units,return_sequences=True, stateful=False,recurrent_initializer='glorot_uniform')(x)
    #x = tf.keras.layers.LSTM(rnn_units,return_sequences=True, stateful=False,recurrent_initializer='glorot_uniform')(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='char_generation_model')
    return model

In [ ]:
model = build_model(embedding_size, rnn_units,vocab_size)


In [ ]:
model.summary()

In [ ]:
# We train on all data, because we don't want to predict but to generate new names
train_batch_ds = data_ds.shuffle(buffer_size=data_size).batch(batch_size,drop_remainder=True)

In [ ]:
input_batch, output_batch = next(iter(train_batch_ds))

In [ ]:
output_batch.numpy().shape, input_batch.numpy().shape
input_batch[:2]

In [ ]:
# we try to predict, even if we have not trained it. it output a shape of batch_size, seq_len (dino_max_len), vocab_size
preds = model.predict(input_batch)
# preds shape is batch_size, seq_len (dino_max_len), vocab_size
pred_names = []
for pred in preds[:1]:
    # We sample from distribution that has been computed by RNN
    sampled_indices = tf.random.categorical(pred, num_samples=1)  
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()  
    pred_names.append(ixs_to_string(sampled_indices))

pred_names

# Training on examples

In [ ]:
# select a loss function
# as our model returns a logit, not a softmax one need to set from_logits=True
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(output_batch, preds)
example_batch_loss

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
import os
import datetime
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

log_dir="logs/rnn-dino-fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,
                                                      embeddings_freq=1)

In [ ]:
history = model.fit(
    train_batch_ds, 
    callbacks=[checkpoint_cb,tensorboard_cb],
    epochs=200)

In [ ]:
tf.debugging.experimental.enable_dump_debug_info("logs/rnn-dino-fit/", tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

In [ ]:
%tensorboard --logdir logs/rnn-dino-fit/

In [ ]:
input_batch, output_batch = next(iter(train_batch_ds))
inputs = input_batch.numpy()
outputs = output_batch.numpy()
input_names = []
output_names = []
for input_idx in inputs[:5]:
    input_names.append(ixs_to_string(input_idx))

for output_idx in outputs[:5]:
    output_names.append(ixs_to_string(output_idx))

input_names, output_names

In [ ]:
input_batch.numpy()

In [ ]:
# do predictions based on training input. this is not name generation, because we provide as input a real name.
preds = model.predict(input_batch)
# preds shape is batch_size, seq_len (dino_max_len), vocab_size
pred_names = []
input_names = []

input_np = input_batch.numpy()

for i, pred in enumerate(preds[:2]) :
    # We sample from distribution that has been computed by RNN
    sampled_indices = tf.random.categorical(pred, num_samples=1)    
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()    
    pred_names.append(ixs_to_string(sampled_indices))
    input_names.append(ixs_to_string(input_np[i]))

input_names, pred_names

# Name generation

![Name generation](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png )

In [ ]:
# we feed into the model one char after the other

def generate_name(model, start_string=None, seq_len=None):
    generated_name = []
    start_ixs = string_to_ixs(start_string.lower())
    input_idx = tf.expand_dims(start_ixs, 0) 
    print(input_idx)   
    #model.reset_states()

    #print(input.shape)
    # essayer en accumulant dans input data le start + les char. 
    for i in range(seq_len):
        pred = model.predict(input_idx)
        
        # remove the batch dimension
        pred = tf.squeeze(pred, 0)
        #print(">", pred)
        #print("pred", pred.shape)
        #print("pred last", tf.random.categorical(pred, num_samples=1).numpy())

        # input : batch_size, num_clas
        predicted_id = tf.random.categorical(pred, num_samples=1)[-1,0].numpy()
        predicted_char = ix_to_char[predicted_id]
        predicted_idx = tf.expand_dims([predicted_id], 0)
        input_idx = tf.concat([input_idx, predicted_idx], axis=1)
        #print(input_idx)
        if predicted_char is '\n':
            break
        generated_name.append(predicted_char)
        
    name = (start_string + ''.join(generated_name))
    return name, len(name)


generate_name(model, start_string="k", seq_len=40)